In [360]:
#installation BeautifulSoup4
!pip install BeautifulSoup4

In [361]:
#installation de parser lxml (c'est un parser plus performant et plus rapide que le parser standard)
!pip install lxml

In [365]:
#mes imports

#pour les appels http
import requests
#pour un pretty print
from pprint import pprint
#pour creer de dataframe
import pandas as pd
#pour le scrapping
from bs4 import BeautifulSoup

In [371]:
start_url = "https://www.automobile-propre.com/voiture-electrique/"
downloaded_html = requests.get(start_url)
pprint(downloaded_html)

<Response [200]>


In [373]:
soup = BeautifulSoup(downloaded_html.text, 'lxml')

In [374]:
# sauvegarder le fichier html sur le disque (si jamais le html sera modifier dans le future), on aura tjs une refrence locale
with open('automobile-propre.html', 'w', encoding="utf-8") as file:
    file.write(downloaded_html.text)

In [415]:
all_result = soup.select('#results_all_vehicles .item_vehicule')
#nbre de voitures électriques trouvés
len(all_result)

151

In [387]:
table_column = ['Titre', 'Lien', 'Prix', 'Description_Prix', 'Modele', 'avis_note', 'nbre_note', 'Autonomie', 'Puissance', 'Recharge', 'Consommation']
pprint(table_column)

['Titre',
 'Lien',
 'Prix',
 'Description_Prix',
 'Modele',
 'avis_note',
 'nbre_note',
 'Autonomie',
 'Puissance',
 'Recharge',
 'Consommation']


In [455]:
table_data = []
for index, element in enumerate(all_result):
    #if index < 1:
        #pprint(index)
        #pprint(element)
        row_list = []
    
        #selecteur de la partie gauche de l'element
        leftSectionSelector = 'div.col-lg-5.col-md-4.col-6'
        leftSection = element.select(leftSectionSelector)[0]     
        #selecteur de la partie droite de l'element
        rightSectionSelector = 'div.col-lg-4.col-md-5.text-right.g-mt-0--md.g-mt-15'
        rightSection = element.select(rightSectionSelector)[0]     
        #selecteur de titre (titre + lien)
        titreSection = leftSection.select('.product-title.g-font-weight-600.ap-roboto.g-font-size-14.g-font-size-16--md.g-mb-2.g-color-black > a')[0]
        #selecteur de prix (prix + description)
        prix_details = leftSection.select('p.g-font-size-14.g-mb-5.g-color-gray-dark-v4')[0]
        #selecteur modele
        modele =  leftSection.select('p.g-color-gray-dark-v5.g-font-size-12.g-mb-0')[0]
        #selecteur notes
        avis_note = leftSection.select('div > div > span:nth-child(2)')
        nbre_avis = leftSection.select('div > div > span:nth-child(3)')
        

        autonomieSelector = ' div:nth-child(1)'
        puissanceSelector = ' div:nth-child(2)'
        rechargeSelector = ' div:nth-child(3)'
        consommationSelector = ' div:nth-child(4)'
        autonomie =  rightSection.select(autonomieSelector)
        puissance =  rightSection.select(puissanceSelector)
        recharge =  rightSection.select(rechargeSelector)
        consommation =  rightSection.select(consommationSelector)
        
        #pprint(autonomie)

        #titre
        row_list.append(titreSection.text)
        #lien
        row_list.append(titreSection.get("href"))
        #prix
        row_list.append(prix_details.text.strip())
        #prix description
        row_list.append(prix_details.select('span')[0].get("title"))
        #modele
        row_list.append(modele.text)
        
        if len(avis_note) > 0:
            row_list.append(avis_note[0].text)
            row_list.append(nbre_avis[0].text.replace('-', ''))
        else:
            row_list.append('No Score')
            row_list.append('No Score')
            
        detailsSelectorIfExist = 'h5.g-color-gray-dark-v3.g-font-weight-400.g-font-size-12.mb-0'
        if len(autonomie) > 0:
            row_list.append(autonomie[0].select(detailsSelectorIfExist)[0].text)
        else:
            row_list.append('NC')
            
        if len(puissance) > 0:
            row_list.append(puissance[0].select(detailsSelectorIfExist)[0].text)
        else:
            row_list.append('NC')     
            
        if len(recharge) > 0:
            row_list.append(recharge[0].select(detailsSelectorIfExist)[0].text)
        else:
            row_list.append('NC')     
            
        if len(consommation) > 0:
            row_list.append(consommation[0].select(detailsSelectorIfExist)[0].text)
        else:
            row_list.append('NC')                     
        
        
        table_data.append(row_list)
        
#print(table_data)
#nbre de lignes
pprint(len(table_data))
#nbre de colonnes
pprint(len(table_data[0]))

151
11


In [453]:
df = pd.DataFrame(table_data, columns=table_column) 
df

,Titre,Lien,Prix,Description_Prix,Modele,avis_note,nbre_note,Autonomie,Puissance,Recharge,Consommation
0,Renault ZOE,https://www.automobile-propre.com/voitures/ren...,32 500 € Hors bonus,À partir de. Hors bonus écologique (de 900 à 7...,Électrique,"4,2",25 avis,171 à 390 km,77 - 135 ch,375 km/h,"17,4 - 17,7"
1,Tesla Model 3,https://www.automobile-propre.com/voitures/tes...,53 490 € Hors bonus,À partir de. Hors bonus écologique (de 900 à 7...,Électrique,"4,4",25 avis,409 à 602 km,275 - 513 ch,1980 km/h,"14,9 - 16,6"
2,Dacia Spring,https://www.automobile-propre.com/voitures/dac...,19 800 € Hors bonus,À partir de. Hors bonus écologique (de 900 à 7...,Électrique,"4,7",6 avis,230 km,44 ch,257 km/h,"13,9"
3,Volkswagen ID.3,https://www.automobile-propre.com/voitures/vol...,39 300 € Hors bonus,À partir de. Hors bonus écologique (de 900 à 7...,Électrique,4,25 avis,350 à 544 km,145 - 204 ch,954 km/h,"13,6 - 17,7"
4,Porsche Taycan,https://www.automobile-propre.com/voitures/por...,109 414 € Hors bonus,À partir de. Hors bonus écologique (de 900 à 7...,Électrique,"4,1",3 avis,407 à 504 km,326 - 761 ch,1626 km/h,"21,1 - 26,7"
...,...,...,...,...,...,...,...,...,...,...,...
146,Opel Movano-e,https://www.automobile-propre.com/voitures/ope...,Prix N.C.\nBientôt,None,Électrique,No Score,No Score,117 à 224 km,122 ch,160 km/h,NC
147,MG 4,https://www.automobile-propre.com/voitures/mg-4/,Prix N.C.,None,Électrique,No Score,No Score,365 à 450 km,170 - 204 ch,949 km/h,160 - 170
148,Fiat e-Scudo,https://www.automobile-propre.com/voitures/fia...,Prix N.C.\nBientôt,None,Électrique,No Score,No Score,230 à 330 km,136 ch,440 km/h,NC
149,Ssangyong Korando E-Motion,https://www.automobile-propre.com/voitures/ssa...,Prix N.C.\nBientôt,None,Électrique,No Score,No Score,339 km,190 ch,661 km/h,NC
